In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
import os
import cv2
from ultralytics import YOLO
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm

# Load the trained model from the checkpoint
checkpoint_path = '/content/drive/MyDrive/aerial_image_recognition/img/train/Tokyo/yolov8_tokyo_checkpoint.pt'
model = YOLO(checkpoint_path)
print("Model loaded from checkpoint successfully!")

# Set up the directories for syncing and storing TIFF files and output
local_tiff_dir = 'G:/Mój dysk/aerial_image_recognition/img/geotiff/'
drive_tiff_dir = '/content/drive/MyDrive/aerial_image_recognition/img/geotiff/'
output_geojson_dir = '/content/drive/MyDrive/aerial_image_recognition/gis/'

# Set the image size (keep it at 1200 pixels)
image_size = 1200

# Initialize an empty GeoDataFrame to store results for the entire area
car_centroids_gdf = gpd.GeoDataFrame(columns=['geometry', 'class', 'confidence'])

# Loop through the chunks in the folder and analyze each TIFF
for tiff_file in os.listdir(local_tiff_dir):
    if tiff_file.endswith('.tiff') or tiff_file.endswith('.tif'):
        # Load the current TIFF file
        tiff_path = os.path.join(local_tiff_dir, tiff_file)
        img = cv2.imread(tiff_path)
        h, w = img.shape[:2]

        print(f"Processing {tiff_file}...")

        # Use a sliding window approach over the entire image (with a 1200x1200 window)
        stride = 600  # Define overlap between windows
        total_tiles = ((h - image_size) // stride + 1) * ((w - image_size) // stride + 1)

        with tqdm(total=total_tiles, desc=f"Processing {tiff_file}") as pbar:
            for y in range(0, h, stride):
                for x in range(0, w, stride):
                    window = img[y:min(y + image_size, h), x:min(x + image_size, w)]

                    # Run the YOLO model on the current window
                    results = model(window)

                    # Process the results (extract bounding boxes, centroids, and confidence)
                    for box in results[0].boxes:
                        if box.cls in [0, 1] and box.conf > 0.4:  # cls=0:Car, cls=1:Truck
                            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                            centroid_x = (x1 + x2) / 2 + x  # Adjust for the window offset
                            centroid_y = (y1 + y2) / 2 + y
                            car_centroids_gdf = car_centroids_gdf.append({
                                'geometry': Point(centroid_x, centroid_y),
                                'class': 'Car' if box.cls == 0 else 'Truck',
                                'confidence': box.conf.item()
                            }, ignore_index=True)

                    pbar.update(1)

        # Optionally delete the TIFF after processing to save space
        os.remove(tiff_path)

# Set CRS to match your area (EPSG:3857 for Web Mercator)
car_centroids_gdf.set_crs(epsg=3857, inplace=True)

# Save the car centroids as a GeoJSON file
output_geojson_path = os.path.join(output_geojson_dir, 'car_centroids.geojson')
car_centroids_gdf.to_file(output_geojson_path, driver='GeoJSON')

print(f"Car and truck centroids saved to {output_geojson_path}")
